# Critical Power 

## 1. Metadata CPs vs Calculated CPs

The aim of this section is to compare the critical power values in the metadata to the calculated values. If they match up well, the metadata measurements can be used instead of having to calculate manually.

In [1]:
# Importing libraries

from opendata import OpenData
import polars as pl
import pandas as pd
import numpy as np

od = OpenData()

In [2]:
# Get a sample athlete's data
athlete = od.get_local_athlete("bce0f962-41e6-4d36-a5f0-e12f6474ea71")


In [3]:
# Using the athlete's fourth activity as a test
activities = list(athlete.activities())
activity = activities[6]

In [4]:
# Storing the critical power values from the metadata in a dictionary
metadata = activity.metadata

keys_to_extract = [
    "1s_critical_power",
    "5s_critical_power",
    "10s_critical_power",
    "15s_critical_power",
    "20s_critical_power",
    "30s_critical_power",
    "1m_critical_power",
    "2m_critical_power",
    "3m_critical_power",
    "8m_critical_power",
    "10m_critical_power",
    "20m_critical_power",
    "30m_critical_power",
    "60m_critical_power",
]

metadata_cp_values = {key: metadata["METRICS"][key] for key in keys_to_extract}
metadata_cp_values

{'1s_critical_power': '213.00000',
 '5s_critical_power': '205.00000',
 '10s_critical_power': '202.00000',
 '15s_critical_power': '201.86667',
 '20s_critical_power': '201.20000',
 '30s_critical_power': '195.90000',
 '1m_critical_power': '187.98333',
 '2m_critical_power': '179.83333',
 '3m_critical_power': '175.41111',
 '8m_critical_power': '173.00833',
 '10m_critical_power': '172.51667',
 '20m_critical_power': '167.57000',
 '30m_critical_power': '159.48500',
 '60m_critical_power': '91.32639'}

In [18]:
metadata

{'date': '2016/02/22 13:06:06 UTC',
 'data': 'TDSP-C-AGL-----',
 'sport': 'Bike',
 'METRICS': {'a_skiba_xpower': ['148.52929', '3703.00000'],
  'a_skiba_relative_intensity': ['0.59412', '3703.00000'],
  'a_skiba_bike_score': '36.30742',
  'a_skiba_variability_index': '1.04745',
  'a_coggan_np': ['152.67347', '3443.00000'],
  'a_coggan_if': ['0.61069', '3443.00000'],
  'a_coggan_tss': '35.66824',
  'a_coggam_variability_index': ['1.07667', '3443.00000'],
  'a_coggan_tssperhour': ['26.76791', '1.33250'],
  'ride_count': '1.00000',
  'workout_time': '4797.00000',
  'time_riding': '3436.00000',
  'time_carrying': '5.00000',
  'total_distance': '29.07951',
  'climb_rating': ['1.00555', '1.00000'],
  'athlete_weight': '58.00000',
  'elevation_gain': '171.00000',
  'elevation_loss': '178.60000',
  'total_work': '488.22100',
  'average_speed': '30.64585',
  'average_power': ['141.80105', '3443.00000'],
  'average_apower': ['141.93024', '3443.00000'],
  'nonzero_power': ['143.08939', '3412.0000

In [ ]:
def calculate_cp(df, window_size):
    """
    Calculate the maximum rolling average power over a specified window size.

    Parameters:
    - df (pl.DataFrame): The input DataFrame containing a 'power' column.
    - window_size (int): The number of rows to include in each rolling window.

    Returns:
    - float: The maximum rolling average power over the specified window size.
    """
    # Calculate the rolling average power for the specified window size
    rolling_avg_power = df.with_columns(
        pl.col("power").rolling_mean(window_size=window_size).alias("rolling_avg_power")
    )

    # Drop null values resulting from the rolling calculation
    rolling_avg_power = rolling_avg_power.drop_nulls(subset=["rolling_avg_power"])

    # Return the maximum rolling average power
    max_rolling_avg_power = rolling_avg_power.select(pl.max("rolling_avg_power")).item()
    return round(float(max_rolling_avg_power), 5)

In [ ]:
# Manually calculating the critical power values for the activity

# Loading the data as a polars dataframe
polars_df = pl.from_pandas(activity.data)

# Checking for continuity in the data using the seconds column
polars_df = polars_df.with_columns(pl.col("secs").diff().alias("delta_secs"))

# Adding a sequence number to each continuous segment of the data
polars_df = polars_df.with_columns(
    (pl.col("delta_secs").ne(1)).cum_sum().alias("sequence_numbers")
)

sequences = polars_df.partition_by("sequence_numbers")[1:]

In [ ]:
# Calculating critical power values for each duration from each sequence

cp_timePoints = [1, 5, 10, 15, 20, 30, 60, 120, 180, 480, 600, 1200, 1800]
calculated_cp_values = {
    "1": [],
    "5": [],
    "10": [],
    "15": [],
    "20": [],
    "30": [],
    "60": [],
    "120": [],
    "180": [],
    "480": [],
    "600": [],
    "1200": [],
    "1800": [],
    "3600": [],
}

for sequence in sequences:
    for timePoint in cp_timePoints:
        if len(sequence) >= timePoint:
            critical_power = calculate_cp(sequence, timePoint)
        else:
            critical_power = np.nan
        calculated_cp_values[str(timePoint)].append(critical_power)

In [13]:
# Storing the maximum power from each nested list in the calculated_cp_values dictionary

for key in calculated_cp_values.keys():
    calculated_cp_values[key].append(np.nan)  # ensuring all lists are non-empty
calculated_cp_values = {key: max(value) for key, value in calculated_cp_values.items()}
calculated_cp_values

{'1': 213.0,
 '5': 205.0,
 '10': 202.0,
 '15': 201.86667,
 '20': 201.2,
 '30': 195.9,
 '60': 187.98333,
 '120': 179.83333,
 '180': 175.41111,
 '480': 173.00833,
 '600': 172.51667,
 '1200': 167.57,
 '1800': 159.485,
 '3600': nan}